In [ ]:
import os
import sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
sys.path.insert(1, r'C:\Users\smartin5\Repos\myNeurochat')
#sys.path.insert(1, r'C:\Users\maolivei\neuro_sean\NeuroChaT')
%load_ext autoreload
%autoreload 2

In [ ]:
from neurochat.nc_spike import NSpike
from neurochat.nc_spatial import NSpatial
from neurochat.nc_data import NData
from neurochat.nc_datacontainer import NDataContainer
import neurochat.nc_plot as nc_plot

In [ ]:
# Setup the main data path
home = os.path.expanduser("~")
data_dir = os.path.join(
    home, "Recordings", "LinearTrack")
prepend_data_dir = lambda name : os.path.join(data_dir, name)

In [ ]:
# Load the data
spike_filenames = [
    "LCA1-linearpresleep30min050319.2",
    "050319D_LCA1_linear_10min.2",
    "LCA1-linearpresleep_40min050319_B.2"]

spike_absolute_filenames = [prepend_data_dir(name) for name in spike_filenames]

spike_names = [
    "pre-sleep",
    "run",
    "post-sleep"]

position_filename = []

collection = NDataContainer(share_positions=True)
collection.add_files(
    #NDataContainer.EFileType.Spike, (spike_absolute_filenames, spike_names, None))
    NDataContainer.EFileType.Spike, spike_absolute_filenames)
collection.load_all_data()

In [ ]:
# Set the correct units for the data
collection.list_all_units()
collection.set_units("all")
print(len(collection))
print(collection)

In [ ]:
# Tests
for data in collection:
    wave_fig = nc_plot.wave_property(data.wave_property())